In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo 

from neural_network import NeuralNetwork

from preprocessing import min_max_scaler, train_test_split
from activation import LeakyRelu, Softmax, Sigmoid, Relu
from loss import CategoricalCrossentropy, BinaryCrossentropy
from optimizer import Adam, SGD
from scheduler import FactorScheduler, CosineScheduler  
from config import NeuralNetworkConfig, TrainingConfig
from regularization import Dropout
from core import ParameterLoader

In [ ]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 

X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

X = X.to_numpy()
y = y.to_numpy(dtype=str)

X, y

In [ ]:
classes = tuple(np.unique(y))

# Normalize data
data = min_max_scaler(X, -1, 1)

# Combine data and labels into a structured array
combined = np.array([(d, label[0]) for d, label in zip(data, y)], dtype=object)

# Split data into training and testing sets
train, test = train_test_split(combined, random_state=0)

classes, train, test

In [ ]:
config = NeuralNetworkConfig(
    network_structure=[len(X[0]), 16, 8, len(classes)],
    classes=classes,
    hidden_activation=LeakyRelu(),
    output_activation=Softmax(),
    loss=CategoricalCrossentropy(),
    optimizer=SGD(momentum=0.9, nesterov=True),
    batch_size=4,
    dropout=Dropout(p=0.1), 
    loader=None
)

nn = NeuralNetwork(config)

nn.train(
    list(train),
    list(test),
    TrainingConfig(
        lr=CosineScheduler(max_steps=100),
        patience_stop=100,
        debug=True,
        store=False,
    ),
)

print(f"final test: {nn.evaluate(list(combined))}, test: {nn.evaluate(list(test))}, train: {nn.evaluate(list(train))}")

In [ ]:
config = NeuralNetworkConfig(
    network_structure=[len(X[0]), 16, 8, 1],
    classes=classes,
    hidden_activation=LeakyRelu(),
    output_activation=Sigmoid(),
    loss=BinaryCrossentropy(),
    optimizer=Adam(),
    batch_size=4,
    dropout=Dropout(p=0.1), 
    loader=None
)

nn = NeuralNetwork(config)

nn.train(
    list(train),
    list(test),
    TrainingConfig(
        lr=CosineScheduler(learning_rate=0.001, min_lr=0.0001, max_steps=20, cyclic=True),
        patience_stop=100,
        debug=True,
        store=False,
    ),
)

print(f"final test: {nn.evaluate(list(combined))}, test: {nn.evaluate(list(test))}, train: {nn.evaluate(list(train))}")